In [1]:
import nltk
modules = ["punkt", "words", "stopwords", "averaged_perceptron_tagger", "maxent_ne_chunker"]
for module in modules:
    nltk.download(module)
    
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from datascience import *
import numpy as np

import locale
import re
import csv
import string
from collections import Counter

[nltk_data] Downloading package punkt to /home/vinitra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/vinitra/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vinitra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vinitra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/vinitra/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [11]:
bechdel = Table.read_table('datasets/2010_2014MoviesTested.csv')
passes_bechdel = bechdel.where(1, 'Yes')
fails_bechdel = bechdel.where(1, 'No')

proportion_of_passes = passes_bechdel.num_rows/bechdel.num_rows
print(str(round(proportion_of_passes*100, 2)) + "% movies from our dataset pass the bechdel test")

63.32% movies from our dataset pass the bechdel test


### Genre Analysis

For the movies that don't pass, do a frequency count on the genres to compare the most common entries

In [31]:
def list_flattening(l):
    return [item for sublist in l for item in sublist]

def clean_genres(table):
    return list_flattening([genre.split(", ") for genre in table['Genre']])
    

In [43]:
f_genres = clean_genres(fails_bechdel)
f_genre_count = Table().with_column('Genre', f_genres).group('Genre').sort('count', descending=True)
f_genre_count = f_genre_count.with_column('f proportion', f_genre_count['count']/sum(f_genre_count['count']))
f_genre_count

Genre,count,f proportion
Drama,248,0.203112
Comedy,188,0.153972
Action,139,0.113841
Thriller,111,0.0909091
Crime,84,0.0687961
Adventure,83,0.0679771
Sci-Fi,58,0.047502
Romance,50,0.04095
Horror,39,0.031941
Animation,38,0.031122


In [47]:
p_genres = clean_genres(passes_bechdel)
p_genre_count = Table().with_column('Genre', p_genres).group('Genre').sort('count', descending=True)
p_genre_count = p_genre_count.with_column('p proportion', p_genre_count['count']/sum(p_genre_count['count']))
p_genre_count

Genre,count,p proportion
Drama,506,0.250495
Comedy,326,0.161386
Romance,149,0.0737624
Thriller,146,0.0722772
Horror,125,0.0618812
Action,119,0.0589109
Adventure,108,0.0534653
Crime,79,0.0391089
Sci-Fi,76,0.0376238
Mystery,63,0.0311881


Let's find the genres with the biggest disparities between passing and not passing!

In [51]:
genre_count = p_genre_count.join("Genre", f_genre_count)
genre_count.append_column('disparity', genre_count['f proportion'] - genre_count['p proportion'])
genre_count.sort('disparity', descending=True)

Genre,count,p proportion,count_2,f proportion,disparity
Action,119,0.0589109,139,0.113841,0.0549302
Crime,79,0.0391089,84,0.0687961,0.0296872
Thriller,146,0.0722772,111,0.0909091,0.0186319
Adventure,108,0.0534653,83,0.0679771,0.0145117
Sci-Fi,76,0.0376238,58,0.047502,0.00987829
War,7,0.00346535,11,0.00900901,0.00554366
Western,1,0.00049505,7,0.00573301,0.00523796
Biography,43,0.0212871,32,0.026208,0.0049209
Sport,7,0.00346535,10,0.00819001,0.00472466
Short,2,0.000990099,5,0.004095,0.00310491


### Plot Keywords

What are the words in the top 10 plot keywords that most indicate a negative bechdel result
    - which words are the most distinctly negative words

### Time Series Analysis

Compare the years of those that passed with those that didn't pass

### Conclusion of Analysis

This is what we think -- 